In [7]:
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import datetime
from tensorflow.keras.layers import Conv2D,Dense,Dropout,Input,GlobalMaxPooling2D,MaxPooling2D,BatchNormalization
%matplotlib inline

In [8]:
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [9]:
train = pd.read_csv('data/sensor_train.csv')
test = pd.read_csv('data/sensor_test.csv')
sub = pd.read_csv('data/提交结果示例.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

In [10]:
train['mod'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['modg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
#train['modtime'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5*train.time_point
#train['modgtime'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5*train.time_point
test['mod'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['modg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5
#test['modtime'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5*test.time_point
#test['modgtime'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5*test.time_point

In [11]:
x = np.zeros((7292, 60, 8, 1))
t = np.zeros((7500, 60, 8, 1))
for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:60]
    x[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:60]
    t[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point'],axis=1), 60, np.array(tmp.time_point))[0]

100%|██████████| 7500/7500 [00:18<00:00, 398.01it/s]


In [15]:
def Net():
    input = Input(shape=(60, 8, 1))
    X = Conv2D(filters=64,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(input)
    X = Conv2D(filters=128,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = Conv2D(filters=256,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = Conv2D(filters=512,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = GlobalMaxPooling2D()(X)
    X = Dropout(0.5)(X)
    X = Dense(19, activation='softmax')(X)
    return tf.keras.models.Model([input], X)

In [14]:
class Acc_combo_Loss(tf.keras.losses.Loss):
    def __init__(self,):
        super(Acc_combo_Loss, self).__init__()
    
    def acc_combo(self,label,prediction):
        # 数值ID与行为编码的对应关系
        mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
            4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
            8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
            12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
            16: 'C_2', 17: 'C_5', 18: 'C_6'}
        # 将行为ID转为编码
        code_y, code_y_pred = mapping[label], mapping[prediction]
        if code_y == code_y_pred: #编码完全相同得分1.0
            return 1.0
        elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
            return 1.0/7
        elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
            return 1.0/3
        else:
            return 0.0
    
    def call(self,y_true,y_pred):
        score = tf.math.cumsum(self.acc_combo(true, pred) for true, pred in zip(y_true, tf.argmax(y_pred,axis=1))) / y_pred.shape[0]
        score=-round(score, 5)
        return score

In [ ]:
kfold = StratifiedKFold(5, shuffle=True)

In [16]:
proba_t = np.zeros((7500, 19))
proba_y = np.zeros((7292, 19))
for fold, (xx, yy) in enumerate(kfold.split(x, y)):
    y_ = tf.keras.utils.to_categorical(y, num_classes=19)
    model = Net()
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['acc'])
    plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_acc",
                                verbose=0,
                                mode='max',
                                factor=0.1,
                                patience=6)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc',
                                   verbose=0,
                                   mode='max',
                                   patience=10)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'fold{fold}.h5',
                                 monitor='val_acc',
                                 verbose=0,
                                 mode='max',
                                 save_best_only=True)
    model.fit(x[xx], y_[xx],
              epochs=100,
              batch_size=512,
              verbose=1,
              shuffle=True,
              validation_data=(x[yy], y_[yy]),
              callbacks=[plateau, early_stopping, checkpoint])
    model.load_weights(f'fold{fold}.h5')
    proba_t += model.predict(t, verbose=0, batch_size=1024) / 5.


Epoch 1/150
12/12 [==============================] - 13s 1s/step - loss: 2.7545 - acc: 0.1785 - val_loss: 2.2788 - val_acc: 0.3502 - lr: 0.0010
Epoch 2/150
12/12 [==============================] - 6s 494ms/step - loss: 2.1154 - acc: 0.3314 - val_loss: 1.9070 - val_acc: 0.3544 - lr: 0.0010
Epoch 3/150
12/12 [==============================] - 6s 496ms/step - loss: 1.8929 - acc: 0.3696 - val_loss: 1.7066 - val_acc: 0.4366 - lr: 0.0010
Epoch 4/150
12/12 [==============================] - 6s 496ms/step - loss: 1.7080 - acc: 0.4236 - val_loss: 1.6242 - val_acc: 0.4387 - lr: 0.0010
Epoch 5/150
12/12 [==============================] - 6s 497ms/step - loss: 1.6053 - acc: 0.4521 - val_loss: 1.5296 - val_acc: 0.4613 - lr: 0.0010
Epoch 6/150
12/12 [==============================] - 6s 496ms/step - loss: 1.5466 - acc: 0.4629 - val_loss: 1.4993 - val_acc: 0.4791 - lr: 0.0010
Epoch 7/150
12/12 [==============================] - 6s 489ms/step - loss: 1.4826 - acc: 0.4850 - val_loss: 1.4580 - val_acc: 

In [17]:
proba_y=model.predict(x, verbose=0, batch_size=1024)
proba_y=np.argmax(proba_y,axis=1)
print(proba_y)

[ 1  1  1 ... 18 18 18]


0.91098

In [ ]:
def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0
score = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y, proba_y)) / proba_y.shape[0]
round(score, 5)

In [19]:
sub.behavior_id = np.argmax(proba_t, axis=1)
from datetime import *
sub.to_csv('data/%s_sub%.5f.csv' %(date.today(),score), index=False)
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   fragment_id  7500 non-null   int64
 1   behavior_id  7500 non-null   int64
dtypes: int64(2)
memory usage: 117.3 KB
